In [1]:
import twint
import nest_asyncio

# Configure
c = twint.Config()
c.Username = "jimcramer"
c.Search = "If"

nest_asyncio.apply()
# Run
twint.run.Search(c)

RefreshTokenException: Could not find the Guest token in HTML

In [13]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('$AMZN (from:jimcramer) since:1990-01-01 until:2023-12-31').get_items()):
    if i>50:
        break
    tweets_list.append([tweet.date, tweet.content, tweet.user.username])
    
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text', 'Username'])

/var/folders/xt/tq9cyssn6k51lc2g0m8nrppc0000gn/T/ipykernel_2980/4086473086.py:11: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets_list.append([tweet.date, tweet.content, tweet.user.username])


In [14]:
tweets_df

,Datetime,Text,Username
0,2021-10-28 08:17:21+00:00,huge club day: $AMZN and $AAPL… stay close …....,jimcramer
1,2021-07-29 20:40:19+00:00,working on $AMZN.. will analysts panic?,jimcramer
2,2021-07-29 20:08:25+00:00,"$AMZN could be nasty, PINS too.. But let's dig...",jimcramer
3,2021-05-11 20:12:27+00:00,"you would think with $AMZN, $SHOP and $OKTA le...",jimcramer
4,2021-04-29 22:26:13+00:00,IGNORE $AMZN AT YOUR PERIL! More on Amazon's b...,jimcramer
5,2021-03-18 09:34:21+00:00,"Yes, it is ridiculous that the ten year treasu...",jimcramer
6,2021-02-24 00:11:01+00:00,at the exact bottom how many people wrote off ...,jimcramer
7,2020-07-02 11:30:44+00:00,Jimmy Chill is sticking with the e-commerce pl...,jimcramer
8,2020-07-02 07:50:29+00:00,IBM has to show several quarters of growth...y...,jimcramer
9,2020-04-28 09:35:44+00:00,Don't confuse realty with the stock market. Yo...,jimcramer
